### Spark Basics: DataFrames and DataSets

There are three different kinds of data modeling primitives that you can use in a Spark application to keep track of transparently distributed collections:  
* RDDs (too low-level to be usable, they are the moral equivalent of assembly language)  
* DataFrames (conceptually inspired by PyData / Pandas, available in Scala and Python)
* DataSets (compile-time type-safe DataFrames, available in Scala but not in Python)

To exemplify a use case for DataFrames and DataSets, the first thing we are going to do is to define some input data by hand.

In [ ]:
val rawData: String = "b,9\nb,3\na,8\nb,2\na,5\na,7\na,9\nb,1"

In [ ]:
val dataAfterSplit = rawData.split("\n")

In [ ]:
dataAfterSplit(0)

You'll notice that each record consists of a text key and a numeric value separated by a comma.  
Next, we define a case class to represent this type of data record.

In [ ]:
case class DataRecord(key: String, value: Int)

We also define a function to parse our raw CSV records into type-safe **`DataRecord`**s.

In [ ]:
def parseIntoDataRecord(s: String): DataRecord = {
  val afterSplit = s.split(",")
  DataRecord( afterSplit(0), afterSplit(1).toInt )
}

Next, we create first a **`DataFrame`** and then a **`DataSet`** based on our data.

In [ ]:
val myFirstDataFrame = sparkSession.createDataFrame( dataAfterSplit.map( parseIntoDataRecord(_) ) )

In [ ]:
val myFirstDataSet = myFirstDataFrame.as[DataRecord]

The data is now in distributed form across the Spark cluster, let's have a look at it.

In [ ]:
myFirstDataSet

We can now work with our data using Spark SQL, which is a SQL-2003 compliant language.  
All we have to do is attach a table name to our **`DataSet`**.

In [ ]:
myFirstDataSet.createOrReplaceTempView("raw_data")

println("Done.")

In [ ]:
sparkSession.sql("SELECT key, SUM(value) AS sum_value FROM raw_data GROUP BY key ORDER BY key").show

In [ ]:
val aggregatedDataFrame = sparkSession.sql("SELECT key, SUM(value) AS sum_value FROM raw_data GROUP BY key ORDER BY key")

In [ ]:
val filteredDataFrame = sparkSession.sql("SELECT * FROM raw_data WHERE value < 7")

In [ ]:
filteredDataFrame

#### Saving to persistent tables

Saving to a persistent table means materializing the contents of the DataFrame and creating a pointer to the data in the Hive metastore.  
This means that the data is saved to a Hive table and the table is registered in the Hive metastore, which means that the table is managed by Hive.

In [ ]:
filteredDataFrame.write.mode("overwrite").saveAsTable("filtered_data")

Later on, when we need this data again, we extract data back out from a Hive table into a different DataFrame.

In [ ]:
val filteredData = sparkSession.table("filtered_data")

In [ ]:
filteredData

#### The Catalog

The catalog is an interface through which we can inspect temp tables as well as Hive-managed tables.

In [ ]:
val c = sparkSession.catalog

In [ ]:
c.listTables.show

#### User Defined Functions (UDFs)

User-Defined Functions (UDFs) is a feature of Spark SQL to define new column-based functions.  
UDFs are very effective because they can be materialized anywhere in the cluster, so the function can be at the same physical location as the data, this makes operating on data using UDFs very efficient.

In [ ]:
// First we define a Scala function
def upCaseSqBr(inputStr: String): String = s"[${inputStr.toUpperCase}]"

println( upCaseSqBr("abc") )

In [ ]:
// Then we register its alias in Spark SQL
sparkSession.udf.register( "upCaseSqBrUDF", upCaseSqBr _ )

In [ ]:
val dataFrameAfterUDF = sparkSession.sql("SELECT upCaseSqBrUDF(key) AS up_case_sq_br, value FROM raw_data")

In [ ]:
dataFrameAfterUDF

A more realistic example of a UDF is one that generates a UUID.

In [ ]:
import java.util.UUID
sparkSession.udf.register("uuid", () => UUID.randomUUID.toString)

In [ ]:
val dataFrameWithUUID = sparkSession.sql("SELECT uuid() AS uuid, * FROM raw_data")

In [ ]:
dataFrameWithUUID

#### Joining Two DataSets

In [ ]:
:sh head -5 /opt/SparkDatasets/geography/cities.csv

In [ ]:
:sh cat /opt/SparkDatasets/geography/cities_header.csv

In [ ]:
case class City (city_id: Long, country_id: Long, city_name: String)

In [ ]:
:sh head -5 /opt/SparkDatasets/geography/countries.csv

In [ ]:
:sh cat /opt/SparkDatasets/geography/countries_header.csv

In [ ]:
case class Country (country_id: Long, continent_id: Long, country_name: String)

In [ ]:
val citySchema = Encoders.product[City].schema

In [ ]:
val countrySchema = Encoders.product[Country].schema

In [ ]:
val citiesDS = sparkSession.read.schema(citySchema).csv("/opt/SparkDatasets/geography/cities.csv").as[City]
citiesDS.createOrReplaceTempView("cities")

In [ ]:
val countriesDS = sparkSession.read.schema(countrySchema).csv("/opt/SparkDatasets/geography/countries.csv").as[Country]
countriesDS.createOrReplaceTempView("countries")

In [ ]:
sparkSession.sql("SELECT countries.country_name, cities.city_name FROM cities INNER JOIN countries ON cities.country_id = countries.country_id")

#### Exercise

Given the population data below, and the assumption that zip codes which are close to each other geographically are also close to each other numerically, compute the top 10 list of the most densely populated "pockets" of zip codes, along with the U.S. state they are in.  
Everything should be type-safe, therefore do use DataSets.  
(Hint: You will need to define a UDF that truncates the 5-digit zip code to 4 digits to find its corresponding pocket.)

In [ ]:
:sh head -5 /opt/SparkDatasets/zipcodes/zips.json

In [ ]:
sparkSession.read.json("/opt/SparkDatasets/zipcodes/zips.json").printSchema

In [ ]:
// Define the case class here

In [ ]:
// Define the schema here

In [ ]:
// Define the temp view here

In [ ]:
// Write the SQL statement here